カメラ観測結果からリンクフローデータを作成

In [1]:
# とりあえず2023/01/29, 2023/01/30の08:00台，17:00台を集計
import pandas as pd 
import numpy as np 
import os 

In [2]:
basepath = '/Users/takahiromatsunaga/res2023/PPcameraTG/camera'
date = '20230130'
df = pd.read_csv(os.path.join(basepath, f'camera_{date}.csv'))

In [6]:
import matplotlib.pyplot as plt 

target_cameras = [1, 2, 3, 4, 5]
grouped = df.groupby('ID_camera')
# dfc_list = [group.reset_index(drop=True) for name, group in grouped]
# for dfc in dfc_list:
#     x1 = dfc['out']
#     x2 = dfc['in']


for name, group in grouped:
    group = group.sort_values(by = 'time').reset_index(drop=True)
    plt.figure()
    plt.plot(group['time'], group['OUT'], label='OUT')
    plt.plot(group['time'], group['IN'], label='IN')
    plt.xlabel('Time')
    plt.ylabel('Count')
    plt.title(f'Camera {name}')
    plt.legend()
    plt.savefig(os.path.join(basepath, f'graph_20230130/camera_{name}.png'))
    plt.close()

In [8]:
# 20分の合計を3/4して15分の通過データに変換する
res_dict = {}
for name, group in grouped:
    out1 = group[group['time'] == '17:30']['OUT'].iloc[0]
    out2 = group[group['time'] == '17:40']['OUT'].iloc[0]

    out_quater = (out1 + out2) * 3 / 4
    res_dict[name] = out_quater

print(res_dict)

{6: 723.75, 7: 606.75, 8: 78.75, 9: 618.75, 12: 1203.0, 13: 1183.5}


In [9]:
# 20分の合計を3/4して15分の通過データに変換する
res_dict = {}
for name, group in grouped:
    out1 = group[group['time'] == '8:30']['OUT'].iloc[0]
    out2 = group[group['time'] == '8:40']['OUT'].iloc[0]

    out_quater = (out1 + out2) * 3 / 4
    res_dict[name] = out_quater

print(res_dict)

{6: 756.75, 7: 561.0, 8: 28.5, 9: 405.0, 12: 663.0, 13: 831.0}


In [10]:
# 20分の合計を3/4して15分の通過データに変換する
res_dict = {}
for name, group in grouped:
    out1 = group[group['time'] == '9:30']['OUT'].iloc[0]
    out2 = group[group['time'] == '9:40']['OUT'].iloc[0]

    out_quater = (out1 + out2) * 3 / 4
    res_dict[name] = out_quater

print(res_dict)

{6: 766.5, 7: 427.5, 8: 23.25, 9: 388.5, 12: 766.5, 13: 744.75}


In [11]:
# 20分の合計を3/4して15分の通過データに変換する
res_dict = {}
for name, group in grouped:
    out1 = group[group['time'] == '13:30']['OUT'].iloc[0]
    out2 = group[group['time'] == '13:40']['OUT'].iloc[0]

    out_quater = (out1 + out2) * 3 / 4
    res_dict[name] = out_quater

print(res_dict)

{6: 447.75, 7: 265.5, 8: 21.0, 9: 382.5, 12: 717.75, 13: 812.25}


抽出したデータから各時間帯のフローデータを作る

In [4]:
import pandas as pd 
df_multi_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/ver0707/multi_node_post.csv')
multi_nodeid_list = sorted(df_multi_node['nodeid'].unique())
flow_dict_origin = {6: 723.75, 7: 606.75, 8: 78.75, 9: 618.75, 12: 1203.0, 13: 1183.5}
flow_dict = {(9, 31): 723.75, (10, 37): 685.75, (11, 34): 618.75, (14, 26): 1203.0, (15, 27): 1183.5}
flow_mat = pd.DataFrame(0, index = multi_nodeid_list, columns = multi_nodeid_list)

# flow_mat

In [5]:
for node_set, flow_value in list(flow_dict.items()):
    k = node_set[0] # nodeid
    a = node_set[1]
    flow_mat.at[k, a] += flow_value


/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_2734/2030752535.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '723.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  flow_mat.at[k, a] += flow_value
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_2734/2030752535.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '685.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  flow_mat.at[k, a] += flow_value
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_2734/2030752535.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '618.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  flow_mat.at[k, 

In [6]:
flow_mat.to_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_17/20230130_1730_flow.csv')

In [4]:
xbounds=-5, 0)*5 + (1, 3)]

In [5]:
xbounds

[(-5, 0, -5, 0, -5, 0, -5, 0, -5, 0, 1, 3)]